In [ ]:
import torch
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timm
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
batch_size = 1
image_size = 224
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Preprocessing for test images
test_transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor()
])

# Load testing data
test_dir = 'images/testing/'
test_data = datasets.ImageFolder(test_dir, transform=test_transform)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Mapping class index to names
class_names = test_data.classes
idx_to_class = {v: k for k, v in test_data.class_to_idx.items()}

In [ ]:
num_classes=2
encoder = timm.create_model(
    'hf_hub:mwalmsley/zoobot-encoder-convnext_nano',
    pretrained=True,
    features_only=True  # gives list of feature maps
)

# Check number of output channels from last feature map
in_channels = encoder.feature_info[-1]['num_chs']
print("Last feature map channels:", in_channels)

# Define custom classifier head
# Define custom classifier head
class CustomHead(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)  # [B, C, H, W] → [B, C, 1, 1]
        self.fc1 = nn.Linear(in_channels, 128)
        self.drop1=nn.Dropout(0.2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    def forward(self, x):
        x = self.global_avg_pool(x)
        x = self.fc1(x)
        x = self.drop1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
# Combine encoder and head
class Classifier(nn.Module):
    def __init__(self, encoder, num_classes):
        super().__init__()
        self.encoder = encoder
        self.head = CustomHead(in_channels, num_classes)
    
    def forward(self, x):
        # encoder returns list of features — take last feature map
        x = self.encoder(x)[-1]
        x = self.head(x)
        return x

model = Classifier(encoder, num_classes)
state_dict = torch.load('best_model.pt', map_location=device)
model.load_state_dict(state_dict)
model = model.to(device)
model.eval()

In [ ]:
# Predictions
all_preds = []
all_probs = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        probs = torch.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        
        all_probs.extend(probs.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

# Predicted class names
predictions = [idx_to_class[k] for k in all_preds]

# Save probabilities and filenames
filenames = [s[0].split(test_dir)[1] for s in test_loader.dataset.samples]
df_percentage = pd.DataFrame(all_probs, columns=class_names)
df_percentage.insert(0, 'name', filenames)
df_percentage['Prediction'] = predictions
df_percentage.to_csv("output.csv", index=False)

# Accuracy
acc = accuracy_score(all_labels, all_preds)
print(f"\nAccuracy: {acc * 100:.2f}%")


# Classification Report
report = classification_report(all_labels, all_preds, target_names=class_names)
print(report)
